##  dataset

https://groups.csail.mit.edu/sls/downloads/restaurant/

In [ ]:
!wget https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttest.bio
!wget https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio

--2022-05-09 05:58:43--  https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttest.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155722 (152K)
Saving to: ‘restauranttest.bio’

restauranttest.bio  100%[===================>] 152.07K   789KB/s    in 0.2s    

2022-05-09 05:58:44 (789 KB/s) - ‘restauranttest.bio’ saved [155722/155722]

--2022-05-09 05:58:44--  https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 759757 (742K)
Saving to: ‘restauranttrain.bio’

restauranttrain.bio 100%[===================>] 741.95K  2.24MB/s    in 0.3s    

2022-05-09 05:58:44 (2.24 MB/s) - ‘restaura

In [ ]:
!pip install spacy==3.0.0
!pip install sklearn_crfsuite
!pip install scikit-learn==0.23.2
!pip install eli5

     |████████████████████████████████| 12.7 MB 5.4 MB/s 
     |████████████████████████████████| 9.1 MB 22.7 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 457 kB 43.0 MB/s 
     |████████████████████████████████| 653 kB 52.7 MB/s 
     |████████████████████████████████| 58 kB 3.3 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uninstall

In [ ]:
# import main libraries
from IPython.display import Image
from IPython.core.display import display, HTML
import pandas as pd
from collections import Counter
import random
from spacy import displacy
import json
import os
from spacy.training import offsets_to_biluo_tags
import spacy
import eli5
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

### Working with the data

Let's have a look at the data. I'll print 30 rows of the dataset.

In [ ]:
with open('restauranttrain.bio', 'r') as f:
    for line in f.readlines()[:30]:
        print(line)

B-Rating	2

I-Rating	start

O	restaurants

O	with

B-Amenity	inside

I-Amenity	dining



O	34



B-Rating	5

I-Rating	star

O	resturants

B-Location	in

I-Location	my

I-Location	town



O	98

B-Restaurant_Name	hong

I-Restaurant_Name	kong

O	restaurant

B-Price	reasonable

O	prices



O	a

O	great

O	lunch

O	spot

O	but

B-Hours	open

I-Hours	till



In [ ]:
# switch column places
with open('restauranttrain.bio', 'r') as f:
    with open('restauranttrain_updated.bio', 'w') as w:
        for line in f.readlines():
            if line == '\n':
                w.write(line)
            else:
                w.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')

In [ ]:
!cat restauranttrain_updated_train.bio

2	B-Rating
start	I-Rating
restaurants	O
with	O
inside	B-Amenity
dining	I-Amenity

34	O

5	B-Rating
star	I-Rating
resturants	O
in	B-Location
my	I-Location
town	I-Location

98	O
hong	B-Restaurant_Name
kong	I-Restaurant_Name
restaurant	O
reasonable	B-Price
prices	O

a	O
great	O
lunch	O
spot	O
but	O
open	B-Hours
till	I-Hours
2	I-Hours
a	I-Hours
m	I-Hours
passims	B-Restaurant_Name
kitchen	I-Restaurant_Name

a	O
place	O
that	O
serves	O
soft	B-Dish
serve	I-Dish
ice	I-Dish
cream	I-Dish

a	O
restaurant	O
that	O
is	O
good	B-Rating
for	B-Amenity
groups	I-Amenity

a	O
salad	B-Dish
would	O
make	O
my	O
day	O

a	O
smoothie	B-Cuisine
would	O
hit	O
the	O
spot	O

a	O
steak	B-Dish
would	O
be	O
nice	O

a	O
sub	B-Dish
place	O
near	B-Location
independence	I-Location
boulevard	I-Location

about	O
how	O
much	O
is	O
a	O
midpriced	B-Price
bottle	B-Rating
of	O
good	B-Rating
wine	B-Cuisine
at	O
davidos	B-Restaurant_Name
italian	I-Restaurant_Name
palace	I-Restaurant_Name

about	O
how	O
much	O
will	O
it	O
cost	B-Pr

In [ ]:
# split into train and validation
count = 0
with open('restauranttrain.bio', 'r') as f:
    with open('restauranttrain_updated_train.bio', 'w') as w1:
        with open('restauranttrain_updated_valid.bio', 'w') as w2:
            for line in f.readlines():
                if count < 6500:
                    if line == '\n':
                        w1.write(line)
                        count += 1
                    else:
                        w1.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')
                else:
                    if line == '\n':
                        w2.write(line)
                    else:
                        w2.write('\t'.join(line.strip().split('\t')[::-1]) + '\n')

In [ ]:
# load the data again

data = []
labels = []
with open('restauranttrain_updated.bio', 'r') as f:
    cur_data = []
    cur_label = []
    for line in f.readlines():
        if line == '\n':
            data.append(cur_data)
            labels.append(cur_label)
            cur_data = []
            cur_label = []
        else:
            cur_data.append(line.strip().split('\t')[0])
            cur_label.append(line.strip().split('\t')[1])

In [ ]:
data[0]

['2', 'start', 'restaurants', 'with', 'inside', 'dining']

In [ ]:
labels[0]

['B-Rating', 'I-Rating', 'O', 'O', 'B-Amenity', 'I-Amenity']

In [ ]:
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')


2	B-Rating
start	I-Rating
restaurants	O
with	O
inside	B-Amenity
dining	I-Amenity


### Rule-based



In [ ]:
predictions = []
for word in data[0]:
    if word.isdigit():
        predictions.append('B-Rating')
    else:
        predictions.append('O')

In [ ]:
f1_score(labels[0], predictions, average='weighted')

0.3571428571428572

### sklearn_crfsuite


In [ ]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(10)

,sent_id,data,entities
0,0,2,B-Rating
1,0,start,I-Rating
2,0,restaurants,O
3,0,with,O
4,0,inside,B-Amenity
5,0,dining,I-Amenity
6,1,34,O
7,2,5,B-Rating
8,2,star,I-Rating
9,2,resturants,O


In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)
sentences = getter.sentences

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i-1][0]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper()
        })
    else:
        features['BOS'] = True
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
X[0][:2]

[{'+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'start',
  'BOS': True,
  'bias': 1.0,
  'word.isdigit()': True,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': '2',
  'word[-2:]': '2',
  'word[-3:]': '2'},
 {'+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:word.lower()': 'restaurants',
  '-1:word.istitle()': False,
  '-1:word.isupper()': False,
  '-1:word.lower()': '2',
  'bias': 1.0,
  'word.isdigit()': False,
  'word.istitle()': False,
  'word.isupper()': False,
  'word.lower()': 'start',
  'word[-2:]': 'rt',
  'word[-3:]': 'art'}]

In [ ]:
X_train = X[:6500]
X_test = X[6500:]
y_train = y[:6500]
y_test = y[6500:]

In [ ]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 6500/6500 [00:00<00:00, 15819.67it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 24417
Seconds required: 0.114

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.22  loss=128815.20 active=24044 feature_norm=1.00
Iter 2   time=0.34  loss=107209.49 active=23652 feature_norm=3.03
Iter 3   time=0.11  loss=93440.08 active=22865 feature_norm=2.33
Iter 4   time=0.12  loss=91345.60 active=23853 feature_norm=2.44
Iter 5   time=0.11  loss=77978.98 active=23148 feature_norm=10.79
Iter 6   time=0.12  loss=54837.49 active=23945 feature_norm=9.94
Iter 7   time=0.11  loss=52297.38 active=23973 feature_norm=10.64
Iter 8   time=0.12  loss=43424.64 active=23801 feature_norm=14.96
Iter 9   time=0.12  loss=36826.02 active=23864 feature_norm=17.58
Iter 10 

In [ ]:
all_entities = sorted(df.entities.unique().tolist())

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9011372104276245

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=[i for i in all_entities if i != 'O'])

0.8208924905397791

In [ ]:
print(metrics.flat_classification_report(y_test, y_pred, labels = all_entities))

                   precision    recall  f1-score   support

        B-Amenity       0.76      0.69      0.72       289
        B-Cuisine       0.85      0.87      0.86       377
           B-Dish       0.82      0.81      0.82       377
          B-Hours       0.85      0.78      0.82       150
       B-Location       0.91      0.88      0.90       649
          B-Price       0.91      0.91      0.91       132
         B-Rating       0.86      0.87      0.86       217
B-Restaurant_Name       0.88      0.82      0.85       200
        I-Amenity       0.69      0.72      0.71       287
        I-Cuisine       0.68      0.77      0.72        82
           I-Dish       0.74      0.70      0.72       196
          I-Hours       0.93      0.89      0.91       196
       I-Location       0.83      0.82      0.82       360
          I-Price       0.75      0.62      0.68        34
         I-Rating       0.68      0.69      0.69        68
I-Restaurant_Name       0.80      0.81      0.80       

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['B-Amenity', 'B-Cuisine', 'B-Dish', 'B-Hours', 'B-Location', 'B-Price', 'B-Rating', 'B-Restaurant_Name', 'I-Amenity', 'I-Cuisine', 'I-Dish', 'I-Hours', 'I-Location', 'I-Price', 'I-Rating', 'I-Restaurant_Name', 'O'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [ ]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
I-Price -> I-Price 6.243081
B-Restaurant_Name -> I-Restaurant_Name 6.080630
I-Dish -> I-Dish  5.980081
I-Rating -> I-Rating 5.857279
B-Price -> I-Price 5.784057
B-Cuisine -> I-Cuisine 5.781400
B-Dish -> I-Dish  5.779397
B-Rating -> I-Rating 5.687013
I-Restaurant_Name -> I-Restaurant_Name 5.573687
B-Location -> I-Location 5.531117
B-Hours -> I-Hours 5.362832
I-Location -> I-Location 5.221802
B-Amenity -> I-Amenity 5.181248
I-Cuisine -> I-Cuisine 5.003985
I-Amenity -> I-Amenity 4.965070
I-Hours -> I-Hours 4.920646
O      -> O       2.621546
O      -> B-Restaurant_Name 2.493133
O      -> B-Location 2.335518
O      -> B-Amenity 2.051701

Top unlikely transitions:
B-Rating -> I-Amenity -2.253132
I-Restaurant_Name -> B-Cuisine -2.276251
B-Dish -> B-Dish  -2.497670
B-Cuisine -> I-Restaurant_Name -2.538116
I-Hours -> B-Hours -2.616475
B-Rating -> B-Rating -2.631814
O      -> I-Rating -2.849268
O      -> I-Dish  -2.944507
I-Amenity -> B-Amenity -3.111448
I-Location -> B-

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
7.181884 B-Location word.lower():closest
6.220792 B-Location word.lower():nearest
6.011736 B-Restaurant_Name word.lower():subway
5.443519 B-Price  +1:word.lower():price
5.333200 B-Price  +1:word.lower():priced
5.304680 I-Cuisine -1:word.lower():sea
5.295840 B-Price  +1:word.lower():prices
5.244082 B-Price  word.lower():cheapest
5.188491 B-Price  word.lower():expensive
5.172428 O        word.lower():and
5.083515 I-Cuisine -1:word.lower():fast
5.049413 B-Restaurant_Name -1:word.lower():does
4.962294 I-Price  +1:word.lower():price
4.771724 O        word.lower():which
4.732549 B-Price  word.lower():affordable
4.688696 B-Location word.lower():downtown
4.502374 B-Cuisine word.lower():seafood
4.453171 B-Amenity word.lower():entertainment
4.435370 B-Rating word.lower():best
4.325131 O        word.lower():have
4.304570 O        word.lower():restaurants
4.263356 B-Price  word.lower():inexpensive
4.225021 B-Rating word.lower():good
4.189484 B-Restaurant_Name word.lower():burgerking


In [ ]:
eli5.show_weights(crf, top=10)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,O,B-Amenity,I-Amenity,B-Cuisine,I-Cuisine,B-Dish,I-Dish,B-Hours,I-Hours,B-Location,I-Location,B-Price,I-Price,B-Rating,I-Rating,B-Restaurant_Name,I-Restaurant_Name
O,2.622,2.052,-4.755,1.503,-3.275,1.796,-2.945,1.48,-4.026,2.336,-4.312,1.646,-3.268,1.882,-2.849,2.493,-3.773
B-Amenity,-0.466,-3.618,5.181,0.08,-1.851,-0.507,-1.247,0.053,-1.383,-0.008,-1.939,-0.007,-1.092,-0.158,-0.95,0.167,-1.915
I-Amenity,0.023,-3.111,4.965,0.433,-1.562,-0.155,-1.223,0.182,-1.76,0.021,-2.055,0.082,-1.27,-0.773,-0.771,0.274,-2.037
B-Cuisine,0.928,0.581,-1.894,-3.663,5.781,0.893,-1.134,-0.144,-0.807,1.1,-1.459,-0.494,-0.668,0.197,-0.687,0.27,-2.538
I-Cuisine,0.483,-1.069,-1.012,-2.187,5.004,0.563,-0.419,-1.116,-0.933,-0.165,-1.349,-0.36,-0.371,0.119,-0.003,-0.721,-1.433
B-Dish,0.0,-0.456,-1.682,-0.48,-1.592,-2.498,5.779,0.308,-1.602,0.629,-1.573,0.225,-0.671,-1.191,-0.476,-1.271,-1.64
I-Dish,0.267,0.754,-1.132,-0.03,-0.604,-1.685,5.98,0.355,-1.085,-0.302,-1.359,0.0,-0.074,-0.307,-0.015,0.0,-0.663
B-Hours,-0.216,0.831,-1.727,-0.34,-0.334,0.012,-0.335,-3.979,5.363,0.604,-1.299,-0.726,-0.453,-0.716,-0.258,-0.854,-0.911
I-Hours,-0.775,0.0,-1.379,0.38,-0.518,-0.359,-0.392,-2.616,4.921,0.357,-1.495,-0.754,-0.276,-0.603,-0.115,-0.334,-0.62
B-Location,-0.019,0.029,-2.005,0.419,-1.659,0.288,-1.3,-0.567,-1.358,-3.735,5.531,-0.054,-1.082,0.248,-1.283,1.541,-2.192


### Spacy

Now let's convert the data into spacy format and train a spacy model.

In [ ]:
!ls 

base_config.cfg      restauranttrain_updated.bio	sample_data
restauranttest.bio   restauranttrain_updated_train.bio
restauranttrain.bio  restauranttrain_updated_valid.bio


In [ ]:
!python -m spacy init config base_config.cfg -p ner


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

⚠ Nothing to auto-fill: base config is already complete
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy convert restauranttrain_updated.bio . -t json -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (1 documents): restauranttrain_updated.json


In [ ]:
!python -m spacy convert restauranttrain_updated_train.bio . -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (6500 documents):
restauranttrain_updated_train.spacy


In [ ]:
!python -m spacy convert restauranttrain_updated_valid.bio . -c ner

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (1160 documents):
restauranttrain_updated_valid.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train restauranttrain_updated_train.spacy --paths.dev restauranttrain_updated_valid.spacy

✔ Created output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
Set up nlp object from config
Pipeline: ['tok2vec', 'ner']
Created vocabulary
Finished initializing nlp object
Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     59.44    0.00    0.00    0.00    0.00
  0     200        248.34   5900.73   55.42   61.62   50.36    0.55
  0     400        480.36   4572.41   68.74   71.09   66.54    0.69
  1     600        645.47   4526.65   73.19   74.67   71.77    0.73
  1     800        773.68   4914.92   77.26   78.39   76.16    0.77
  2    1000        948.89   5124.43   77.36   78.82   75.95    0.77
  3   

In [ ]:
!python -m spacy evaluate output/model-last restauranttrain_updated_valid.spacy

ℹ Using CPU

================================== Results ==================================

TOK     -    
NER P   80.30
NER R   77.04
NER F   78.63
SPEED   13909


=============================== NER (per type) ===============================

                      P       R       F
Rating            74.54   74.19   74.36
Dish              74.42   76.39   75.39
Cuisine           82.38   80.64   81.50
Location          87.93   83.05   85.42
Hours             81.82   66.00   73.06
Price             91.13   85.61   88.28
Amenity           69.96   63.67   66.67
Restaurant_Name   76.62   77.00   76.81



In [ ]:
nlp = spacy.load("output/model-best")
for i in range(10, 20):
    doc = nlp(' '.join(data[i]))

    spacy.displacy.render(doc, style="ent", jupyter=True)